<a href="https://colab.research.google.com/github/gupta24789/multiclass-classification/blob/main/multiclass_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 18.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
import torchmetrics
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

## Set Seed

In [3]:
SEED = 121
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.seed_everything(SEED)

INFO:lightning_fabric.utilities.seed:Seed set to 121


121

## Load Read Data

In [4]:
train_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/multiclass-classification/main/data/train.txt", header = None, sep=';')
val_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/multiclass-classification/main/data/val.txt",header = None, sep=';')
train_df.columns = ['text','label']
val_df.columns = ['text','label']

In [5]:
print(train_df.label.value_counts())

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: label, dtype: int64


In [6]:
print(val_df.label.value_counts())

joy         704
sadness     550
anger       275
fear        212
love        178
surprise     81
Name: label, dtype: int64


In [7]:
train_df.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


## Encode Label

In [8]:
## Encode Labels
label2idx_map = {w:i for i,w in enumerate(train_df.label.unique().tolist())}
idx2label_map = {i:w for w,i in label2idx_map.items()}

train_df['encoded_label'] = train_df.label.apply(lambda x: label2idx_map[x])
val_df['encoded_label'] = val_df.label.apply(lambda x: label2idx_map[x])

## Class Weight

In [9]:
class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(train_df.encoded_label), y=train_df.encoded_label)
class_weights

array([0.57151022, 1.23513973, 2.04498978, 4.66200466, 1.37669936,
       0.49732687])

## Transformer Model Exploration

In [168]:
model_name = "albert-base-v2"
# model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = AutoModel.from_pretrained(model_name)

In [169]:
inputs = tokenizer("I hate you", return_tensors='pt')
inputs

{'input_ids': tensor([[   2,   31, 3223,   42,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [170]:
embedding = transformer_model(**inputs)

In [171]:
last_hidden_state, pooler_output = embedding['last_hidden_state'], embedding['pooler_output']

In [172]:
last_hidden_state.shape, pooler_output.shape

(torch.Size([1, 5, 768]), torch.Size([1, 768]))

In [173]:
## last hidden state
last_hidden_state[:,0,:].shape

torch.Size([1, 768])

In [174]:
## average of last 4 hidden states
torch.mean(last_hidden_state[:,-4:,:], dim = 1).shape

torch.Size([1, 768])

## Data Loaders

In [175]:
train_df.text.str.split(" ").str.len().describe([.99])

count    16000.000000
mean        19.166313
std         10.986905
min          2.000000
50%         17.000000
99%         52.000000
max         66.000000
Name: text, dtype: float64

In [176]:
def custom_collate(batch):

  text = [item['text'] for item in batch]
  label = [item['encoded_label'] for item in batch]

  inputs = tokenizer(text, max_length= 60, truncation=True, padding='max_length', return_tensors='pt')
  label = torch.tensor(label, dtype = torch.long)

  batch = {"input_ids": inputs['input_ids'], "token_type_ids": inputs['token_type_ids'],"attention_mask": inputs['attention_mask'], "label": label}
  return batch

In [177]:
train_data = train_df[['text','encoded_label']].to_dict('records')
val_data = val_df[['text','encoded_label']].to_dict('records')

In [178]:
train_data[:2]

[{'text': 'i didnt feel humiliated', 'encoded_label': 0},
 {'text': 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'encoded_label': 0}]

In [179]:
batch_size = 2
train_dl = DataLoader(train_data, batch_size = batch_size, shuffle = True, collate_fn= custom_collate)

In [180]:
example = next(iter(train_dl))
example['input_ids'].shape, example['token_type_ids'].shape, example['attention_mask'].shape, example['label'].shape

(torch.Size([2, 60]),
 torch.Size([2, 60]),
 torch.Size([2, 60]),
 torch.Size([2]))

In [181]:
example['input_ids']

tensor([[    2,    31,  3538,    20,   583, 27729,    21,   746, 14057,  7775,
            13,  6483,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    2,    31,  5480,    20,    57,    21, 19937,    68,  1249,    54,
           913,   164, 13654,    47,    30,  1259,    16,  6691,   104,  1533,
            70,    14,   161,    14,   360,    25,   142, 10394,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])

In [182]:
example['token_type_ids']

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [183]:
example['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [184]:
example['label']

tensor([5, 0])

In [185]:
## dataloaders
batch_size = 64
train_dl = DataLoader(train_data, batch_size = batch_size, shuffle = True, collate_fn= custom_collate, num_workers = 2)
val_dl = DataLoader(val_data, batch_size = batch_size, shuffle = False, collate_fn= custom_collate, num_workers = 2)

## Build Model

In [194]:
class MultiClassTransformer(pl.LightningModule):

  def __init__(self, output_dim, learning_rate, dropout, freeze = False):
    super().__init__()
    self.learning_rate = learning_rate

    ## define loss & accuracy
    self.loss_fn = nn.CrossEntropyLoss()
    # self.loss_fn = nn.CrossEntropyLoss(weight= torch.tensor(class_weights, dtype = torch.float))
    self.train_f1 = torchmetrics.F1Score(task="multiclass", num_classes=output_dim)
    self.val_f1 = torchmetrics.F1Score(task="multiclass", num_classes=output_dim)

    ## define layers
    self.transformer_model = AutoModel.from_pretrained(model_name)
    hidden_dim = self.transformer_model.config.hidden_size
    self.linear1 = nn.Linear(hidden_dim, 256)
    self.linear2 = nn.Linear(256, output_dim)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(dropout)

    ## freeze layers
    bertLayerList = ['pooler.weight','pooler.bias']
    if freeze:
      for name, params in self.transformer_model.named_parameters():
        if name not in bertLayerList:
          params.requires_grad = False


  def forward(self, inputs):
    """
    No need to apply softmax at the end as crossentropy implicitly apply the softmax
    """
    embeddings = self.transformer_model(**inputs)
    last_hidden_state, pooler_output = embeddings['last_hidden_state'], embeddings['pooler_output']

    ## last hidden state
    # hidden_state = last_hidden_state[:,0,:]
    ## pooler state
    # hidden_state = pooler_output
    ## average of last 4 hidden state
    hidden_state = torch.mean(last_hidden_state[:,-4:,:], dim = 1)


    out = self.relu(self.linear1(hidden_state))
    out = self.dropout(out)
    out = self.linear2(out)
    return out

  def _shared_step(self, batch):
    label = batch.pop('label')
    logits = self(batch)
    loss = self.loss_fn(logits, label)
    return logits, loss, label

  def training_step(self, batch, batch_idx):
    logits, loss, label = self._shared_step(batch)
    self.train_f1.update(logits, label)
    self.log_dict({"train_loss": loss, "train_f1": self.train_f1}, on_step = False, on_epoch = True, prog_bar=True)
    return loss

  def validation_step(self,batch, batch_idx):
    logits, loss, label = self._shared_step(batch)
    self.val_f1.update(logits, label)
    self.log_dict({"val_loss": loss,  "val_f1": self.val_f1}, on_step = False, on_epoch = True, prog_bar=True)
    return loss

  def on_training_epoch_end(self):
    self.train_f1.reset()

  def on_validation_epoch_end(self):
    if self.current_epoch!=0:
      print(f"Epoch : {self.current_epoch} Val F1 : {self.val_f1.compute()}")
    self.val_f1.reset()

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

In [195]:
# ## test model architecture
# model = MultiClassTransformer(output_dim = len(label2idx_map), learning_rate = 1e-3, freeze = False)
# inputs = {
#     "input_ids": example['input_ids'],
#     "token_type_ids": example['token_type_ids'],
#     "attention_mask": example['attention_mask']
# }
# logits = model(inputs)
# model.loss_fn(logits, example['label'])

In [196]:
## Model Training

model = MultiClassTransformer(output_dim = len(label2idx_map), learning_rate = .0001, dropout = 0.2, freeze = True)

callbacks = pl.callbacks.ModelCheckpoint(dirpath = "multiclass_logs",
                                         filename = '{epoch}-{val_loss:.2f}-{val_f1:.2f}',
                                          mode = "max",
                                          monitor = "val_f1",
                                          save_last = True,
                                          save_top_k=-1)

trainer = pl.Trainer(accelerator= "gpu",
           max_epochs=5,
           check_val_every_n_epoch = 1,
           callbacks = [callbacks])

trainer.fit(model, train_dl, val_dl)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/multiclass_logs exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type              | Params
--------------------------------------------------------
0 | loss_fn           | CrossEntropyLoss  | 0     
1 | train_f1          | MulticlassF1Score | 0     
2 | val_f1            | MulticlassF1Score | 0     
3 | transformer_model | AlbertModel       | 11.7 M
4 | linear1           | Linear            | 196 K 
5 | linear2       

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch : 1 Val F1 : 0.40700000524520874


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch : 2 Val F1 : 0.4115000069141388


## Load Model

In [112]:
checkpoint = "multiclass_logs/epoch=4-val_loss=1.43-val_acc=0.00.ckpt"

In [121]:
# ## Load model in cpu
# model = MultiClassTransformer.load_from_checkpoint(checkpoint,
#                                             output_dim = len(label2idx_map),
#                                            learning_rate = 1e-3,
#                                            freeze = False,
#                                            dropout = 0.1,
#                                            map_location = "cpu")

In [128]:
# model = MultiClassTransformer.load_from_checkpoint(checkpoint,
#                                                    output_dim = len(label2idx_map),
#                                                    learning_rate = 1e-3,
#                                                    freeze = False,
#                                                    dropout = 0.1,
#                                                    map_location = "cuda")

In [129]:
model = model.eval()

In [136]:
def predict(text):
  inputs = tokenizer(text, return_tensors='pt')
  inputs.to('cuda:0')
  preds = model(inputs)
  value, index = torch.topk(preds.cpu(), k = 1)
  return index.item()

In [137]:
index = predict("I love you")
print(f"Label : {idx2label_map[index]}")

Label : joy


In [138]:
index = predict("i hate you")
print(f"Label : {idx2label_map[index]}")

Label : sadness


## Classification report

In [139]:
val_preds_index = [predict(text) for text in val_df.text]

In [140]:
print(classification_report(val_df.encoded_label, val_preds_index, target_names = list(label2idx_map.keys())))

              precision    recall  f1-score   support

     sadness       0.42      0.73      0.54       550
       anger       0.00      0.00      0.00       275
        love       0.00      0.00      0.00       178
    surprise       0.00      0.00      0.00        81
        fear       0.00      0.00      0.00       212
         joy       0.56      0.83      0.67       704

    accuracy                           0.49      2000
   macro avg       0.16      0.26      0.20      2000
weighted avg       0.31      0.49      0.38      2000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [141]:
val_df.label.value_counts()

joy         704
sadness     550
anger       275
fear        212
love        178
surprise     81
Name: label, dtype: int64